# UT10: Proyecto Final Integrador

Análisis de datos con Python

# UT10: Proyecto Final Integrador

> **Cuaderno de trabajo — UT10: Proyecto Final Integrador**
>
> Este notebook contiene las instrucciones y la autoevaluación del
> proyecto. Para el contexto completo y la rúbrica consulta el libro
> (PDF).

## El *dataset*: House Prices

Trabajarás con una versión del famoso *dataset* “Ames Housing”.

-   **Fichero:** `house_prices_raw.csv` (disponible en la carpeta
    `data/`).
-   **Registros:** 1460 viviendas vendidas.
-   **Variables:** 79 columnas describiendo casi cada aspecto de la
    casa.
-   **Target:** `SalePrice` (precio de venta en dólares).

**Advertencia de calidad:** el *dataset* es “real”. Contiene nulos,
errores tipográficos, variables categóricas con muchas clases y
*outliers*. No intentes meterlo en un modelo sin procesarlo, o fallará.

## Ingeniería de datos (la base)

**Objetivo:** crear un «Golden Dataset» limpio y procesado listo para el
análisis.

En UT2 aprendiste operaciones vectorizadas con NumPy: la base de los
arrays que utilizan internamente todos los modelos de *Machine
Learning*. En UT3, la manipulación de DataFrames con Pandas: carga,
filtrado, agrupación y transformación de datos. Ambas habilidades son la
base de toda la ingeniería de datos de este proyecto.

En UT4 aprendiste a cargar datos desde múltiples fuentes (CSV, JSON,
APIs). En UT5, las técnicas de limpieza e imputación. Aquí las aplicarás
a un *dataset* real con problemas reales. Si necesitas repasar, revisa
las secciones de imputación de UT5 antes de empezar.

### Auditoría de calidad

Lo primero es entender qué está roto. Crea un *notebook*
`01_limpieza.ipynb`.

-   Carga los datos.
-   Identifica columnas con $> 40\ \%$ de nulos. ¿Merece la pena
    mantenerlas?
-   Identifica inconsistencias en variables categóricas (ej:
    `GarageYrBlt` futuro).

**Atajo profesional:** usa `df.info()` y
`df.isnull().mean().sort_values(ascending=False).head(15)` para obtener
una radiografía rápida del *dataset*. En proyectos reales, las
bibliotecas como `ydata-profiling` generan informes de calidad
automáticos con una sola línea.

### Estrategia de imputación (referencia UT5)

No puedes simplemente hacer `.dropna()`. Perderías demasiados datos.

-   **Numéricas:** ¿media o mediana? (Pista: mira la distribución de
    `LotFrontage`).
-   **Categóricas:** ¿moda o una nueva categoría “Sin Información”?.

Para detectar distribuciones sesgadas, el rango intercuartílico es tu
aliado:

$IQR = Q_3 - Q_1$

Si un valor está por debajo de $Q_1 - 1.5 \cdot IQR$ o por encima de
$Q_3 + 1.5 \cdot IQR$, se considera *outlier*. Usa la mediana en lugar
de la media cuando haya valores extremos.

### *Feature engineering* básico

Crea variables que aporten valor:

-   `TotalSF` = `1stFlrSF` + `2ndFlrSF` + `TotalBsmtSF` (superficie
    total real).
-   `HouseAge` = `YrSold` - `YearBuilt` (edad al venderse).

**Error frecuente — *data leakage* en *feature engineering***: nunca
uses `SalePrice` para construir nuevas variables (ej:
`PrecioPorM2 = SalePrice / TotalSF`). Estarías “filtrando” la respuesta
dentro de los datos de entrada, y tu modelo parecerá perfecto pero será
inútil en producción. Toda *feature* nueva debe construirse **solo** con
variables predictoras.

**Entregable del hito 1:**

un archivo `house_prices_clean.csv` sin valores nulos y con las nuevas
variables creadas.

## Análisis exploratorio (el detective)

**Objetivo:** entender qué mueve el mercado inmobiliario. Crea
`02_eda.ipynb`.

En UT6 aprendiste el flujo EDA completo: univariante, bivariante y
multivariante. En UT7, las técnicas de visualización avanzada con
Matplotlib y Seaborn. Aquí aplicarás ambas para descubrir qué variables
realmente importan para predecir el precio.

### Análisis univariante (*target*)

Analiza `SalePrice`.

-   ¿Sigue una distribución normal?
-   Haz un histograma y un *boxplot*.
-   **Pro-tip:** los precios suelen tener sesgo positivo (cola derecha).
    Prueba a aplicar `np.log1p()` y visualiza de nuevo.

**Atajo profesional:** la transformación logarítmica $\log(1 + x)$ es
estándar en precios inmobiliarios. Convierte una distribución sesgada en
algo más simétrico, lo que mejora el rendimiento de modelos lineales. En
el mundo real, muchas variables económicas (salarios, precios, ingresos)
se analizan en escala logarítmica.

### Análisis bivariante (correlaciones)

¿Qué importa más?

-   Crea una matriz de correlación (*heatmap*) con las variables
    numéricas.
-   Identifica las 5 variables con mayor correlación con `SalePrice`.
-   Visualiza la relación `GrLivArea` vs. `SalePrice` con un
    *scatterplot*.

**Error frecuente — correlación no es causalidad:** que dos variables
tengan un coeficiente de Pearson alto ($r > 0.7$) no significa que una
cause la otra. Por ejemplo, `GarageCars` y `GarageArea` correlacionan
fuertemente entre sí; pero incluir ambas en el modelo añade
multicolinealidad sin mejorar la predicción. Selecciona la más
representativa.

### Detección de *outliers*

Busca casas con precios sospechosos.

-   ¿Hay casas enormes con precios ridículamente bajos? (Posibles ventas
    familiares o errores).
-   Usa el criterio IQR o visualización para filtrar casos extremos que
    puedan confundir al modelo.

**Cuidado al eliminar *outliers***: eliminar datos es una decisión que
debe justificarse. No borres filas solo porque “se ven raras”. Documenta
cuántos registros eliminas, por qué criterio y qué porcentaje del
*dataset* representan. En la rúbrica se valora la justificación, no la
cantidad eliminada.

**Entregable del hito 2:** un informe en Markdown (dentro del
*notebook*) con los «Top 5 Drivers del Precio» y 3 gráficos clave
exportados a PNG.

## Modelado predictivo (el cerebro)

**Objetivo:** entrenar un algoritmo capaz de generalizar. Crea
`03_modelado.ipynb`.

En UT8 aprendiste el flujo completo de ML supervisado: preprocesamiento,
*split*, entrenamiento y evaluación. Aquí lo aplicarás a un problema
real de regresión. Revisa la sección de `train_test_split` y
`cross_val_score` en UT8 si necesitas refrescar conceptos.

### Preprocesamiento para ML (referencia UT8)

Las máquinas no entienden texto.

-   Aplica ***one-hot encoding*** a las variables categóricas (ej:
    `Neighborhood`).
-   Divide en ***train/test split*** (80/20). ¡No olvides la semilla
    aleatoria!

**El error número 1 en proyectos ML — *data leakage***: el *split*
*train/test* debe hacerse **antes** de cualquier transformación
(escalado, *encoding*, imputación). Si ajustas un `StandardScaler` sobre
todo el *dataset* y luego divides, las estadísticas del *test*
“contaminan” el *train*. Usa `Pipeline` de scikit-learn para encadenar
pasos de forma segura.

### Entrenamiento

Entrena al menos dos modelos para comparar:

1.  **Regresión lineal:** tu *baseline*. Simple e interpretable.
2.  ***Random Forest Regressor***: modelo avanzado capaz de capturar no
    linealidades.

**Atajo profesional:** empieza siempre con un modelo simple
(*baseline*). Si tu regresión lineal ya consigue $R^2 = 0.75$, sabes que
el problema es resoluble y tienes una referencia para medir mejoras. En
la industria, un modelo simple que funciona es mejor que uno complejo
que no se entiende.

### Evaluación

No te fíes del *accuracy*. Estamos en regresión. Las métricas clave son:

$RMSE = \sqrt{\frac{1}{n}\sum_{i=1}^{n}(y_i - \hat{y}_i)^2}$

El RMSE te dice cuántos dólares te equivocas de media. Tiene las mismas
unidades que el *target*.

$R^2 = 1 - \frac{\sum_{i=1}^{n}(y_i - \hat{y}_i)^2}{\sum_{i=1}^{n}(y_i - \bar{y})^2}$

El $R^2$ indica qué proporción de la varianza del precio explica tu
modelo. Un $R^2 = 0.85$ significa que tu modelo captura el 85 % de la
variabilidad.

## Conceptos clave

-   **Serialización:** proceso de guardar un modelo de *Machine
    Learning* entrenado (ej: archivo `.pkl`) para poder usarlo
    posteriormente en una aplicación.
-   **MVP (*Minimum Viable Product*):** versión inicial de un proyecto
    que incluye solo las funciones esenciales para ser funcional y
    validable.
-   ***Data Leakage***: error grave donde información del conjunto de
    *test* se “filtra” en el entrenamiento, falseando los resultados.

**Criterio de éxito:** tu modelo *Random Forest* debería obtener un
$R^2 > 0.80$ en el conjunto de *test*. Si obtienes $R^2 > 0.98$,
sospecha: es probable que haya *data leakage*.

### Guardado

Usa la librería `joblib` o `pickle` para guardar tu modelo entrenado
(`modelo_final.pkl`). Lo necesitarás para la app.

## Producto final (la app)

**Objetivo:** convertir tu modelo en una herramienta usable por humanos.

Crea un script `app.py` usando **Streamlit** (referencia UT9).

En UT9 aprendiste a crear *dashboards* interactivos con Streamlit:
*widgets* de entrada, *layouts*, `st.metric()`, gráficos embebidos. Aquí
conectarás todo con un modelo real entrenado por ti. Revisa los ejemplos
de `st.sidebar` y `st.selectbox` si los necesitas.

### Requisitos de la app:

1.  **Título y contexto:** explica qué hace la herramienta.
2.  **Entradas (*sidebar*):** permite al usuario introducir datos de una
    casa:
    -   Metros cuadrados (*slider*).
    -   Número de baños (*number input*).
    -   Calidad general (*selectbox*).
    -   Barrio (*selectbox*).
3.  **Botón «tasar»:** al hacer clic, carga el modelo `.pkl` y lanza la
    predicción.
4.  ***Output* visual:** muestra el precio estimado en grande y quizás
    un gráfico comparativo.

In [1]:
# Snippet de ayuda para cargar modelo
import joblib
modelo = joblib.load('modelo_final.pkl')
prediction = modelo.predict(input_data)
st.metric("Precio Estimado", f"${prediction[0]:,.2f}")

**Error frecuente — rutas *hardcodeadas***: no uses rutas absolutas como
`C:\Users\tu_nombre\modelo.pkl`. Usa rutas relativas
(`./modelo_final.pkl`) o `pathlib.Path`. Tu app debe funcionar en
cualquier máquina sin cambiar el código. Incluye un `requirements.txt`
con las dependencias exactas.

**Atajo profesional:** antes de desplegar, prueba tu app con entradas
extremas. ¿Qué pasa si alguien pone 0 m²? ¿O 50 baños? Un `try/except`
con un mensaje amigable (`st.error("Valor fuera de rango")`) marca la
diferencia entre una app de estudiante y una profesional.

**En la entrevista de trabajo:** el proyecto que acabas de completar es
un *portfolio piece*. Las empresas de datos valoran más un proyecto
*end-to-end* bien explicado que diez ejercicios sueltos. Cuando
presentes tu trabajo en una entrevista, enfatiza las **decisiones** que
tomaste (por qué mediana en vez de media, por qué *Random Forest*, cómo
validaste sin *data leakage*) más que el resultado numérico.

**Consejo final:** no busques el modelo perfecto con 0.99 de $R^2$. En
el mundo real vale más un modelo de 0.85 bien explicado, robusto y
desplegado en una app útil que un modelo de 0.99 que vive muerto en un
*notebook*.